In [ ]:
import os
import safetensors
import torch
import json

def check_cached_latent(path):
    try:
        with safetensors.safe_open(path, framework="pt", device="cpu") as f:
            metadata = f.metadata()
            keys = f.keys()
            
            print(f"File: {os.path.basename(path)}")
            print(f"Metadata present: {'Yes' if metadata else 'No'}")
            print(f"Keys in the safetensor: {', '.join(keys)}")
            
            if 'latent' in keys:
                latent = f.get_tensor('latent')
                print(f"Latent shape: {latent.shape}")
                print(f"Latent dtype: {latent.dtype}")
            
            if metadata:
                print("\nMetadata:")
                for key, value in metadata.items():
                    try:
                        parsed_value = json.loads(value)
                        print(f"  {key}: {parsed_value}")
                    except json.JSONDecodeError:
                        print(f"  {key}: {value}")
            
    except Exception as e:
        print(f"Error processing the cached latent file: {str(e)}")

def scan_latent_cache(cache_dir):
    print(f"Scanning latent cache directory: {cache_dir}")
    for root, _, files in os.walk(cache_dir):
        for file in files:
            if file.endswith('.safetensors'):
                full_path = os.path.join(root, file)
                print("\n" + "="*50)
                check_cached_latent(full_path)

#scan file instead of directory
def scan_latent_file(file_path):
    print(f"Scanning latent file: {file_path}")
    check_cached_latent(file_path)

# Example usage
if __name__ == "__main__":
    file_path = r"P:\downloads\latents\_latent_cache\img_028_bLPQW46Jb_ve7lfwu4HXEA.safetensors"
    scan_latent_file(file_path)

In [ ]:
import os
import torch
from diffusers import AutoencoderTiny
import safetensors
from PIL import Image
import matplotlib.pyplot as plt

def load_vae(device='cuda' if torch.cuda.is_available() else 'cpu'):
    vae = AutoencoderTiny.from_pretrained("madebyollin/taef1", torch_dtype=torch.bfloat16).to(device)
    return vae

def load_latent(path):
    try:
        with safetensors.safe_open(path, framework="pt", device="cpu") as f:
            latent = f.get_tensor('latent')
        return latent
    except Exception as e:
        print(f"Error loading latent from {path}: {str(e)}")
        return None

def decode_latents(latents: torch.Tensor, vae: AutoencoderTiny):
    latents = latents.to(vae.device, dtype=torch.bfloat16)
    latents = latents / vae.config.scaling_factor
    with torch.no_grad():
        images = vae.decode(latents).sample
    return images

def show_images(images, name='Decoded Images'):
    plt.figure(figsize=(12, 12))
    plt.suptitle(name, fontsize=16)
    for i, img_tensor in enumerate(images):
        img = tensor_to_pil(img_tensor)
        plt.subplot(1, len(images), i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def tensor_to_pil(img):
    img = (img / 2 + 0.5).clamp(0, 1)
    img = img.cpu().float().permute(1, 2, 0).numpy()
    return Image.fromarray((img * 255).astype("uint8"))

def process_latent_file(file_path, vae):
    print(f"Processing: {file_path}")
    latent = load_latent(file_path)
    if latent is not None:
        decoded_images = decode_latents(latent.unsqueeze(0), vae)
        show_images(decoded_images, name=f"Decoded: {os.path.basename(file_path)}")

def process_directory(directory, vae):
    for file in os.listdir(directory):
        if file.endswith('.safetensors'):
            process_latent_file(os.path.join(directory, file), vae)

if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    vae = load_vae(device)
    
    latent_file = r"G:\comfy_dev\ComfyUI\input\img_001_MxN4PuRrT4mg5FmEPGDf8g.safetensors"
    process_latent_file(latent_file, vae)
    


In [ ]:
import os
import torch
from diffusers import AutoencoderTiny
import safetensors
from PIL import Image
import matplotlib.pyplot as plt

def load_vae(device='cuda' if torch.cuda.is_available() else 'cpu'):
    vae = AutoencoderTiny.from_pretrained("madebyollin/taef1", torch_dtype=torch.bfloat16).to(device)
    return vae

def load_latent(path):
    try:
        with safetensors.safe_open(path, framework="pt", device="cpu") as f:
            latent = f.get_tensor('latent')
        return latent
    except Exception as e:
        print(f"Error loading latent from {path}: {str(e)}")
        return None

def decode_latents(latents: torch.Tensor, vae: AutoencoderTiny):
    latents = latents.to(vae.device, dtype=torch.bfloat16)
    latents = latents / vae.config.scaling_factor
    with torch.no_grad():
        images = vae.decode(latents).sample
    return images

def tensor_to_pil(img):
    img = (img / 2 + 0.5).clamp(0, 1)
    img = img.cpu().float().permute(1, 2, 0).numpy()
    return Image.fromarray((img * 255).astype("uint8"))

def process_latent_file(file_path, vae, output_dir):
    print(f"Processing: {file_path}")
    latent = load_latent(file_path)
    if latent is not None:
        decoded_images = decode_latents(latent.unsqueeze(0), vae)
        for i, img_tensor in enumerate(decoded_images):
            img = tensor_to_pil(img_tensor)
            output_filename = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(file_path))[0]}_{i}.png")
            img.save(output_filename)
            print(f"Saved: {output_filename}")

def process_directory(input_dir, vae):
    output_dir = os.path.join(input_dir, "decoded")
    os.makedirs(output_dir, exist_ok=True)
    
    for file in os.listdir(input_dir):
        if file.endswith('.safetensors'):
            process_latent_file(os.path.join(input_dir, file), vae, output_dir)

if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    vae = load_vae(device)
    
    input_dir = r"P:\downloads\latents (5)\imagecache"
    process_directory(input_dir, vae)

In [ ]:
import os
from PIL import Image
import sys
from PIL.ExifTags import TAGS
from IPython.display import display, Image as IPImage

def read_png_metadata(file_path):
    try:
        with Image.open(file_path) as img:
            # Display the image
            display(IPImage(filename=file_path))
            
            # Check if it's a PNG file
            if img.format != 'PNG':
                print(f"Error: {file_path} is not a PNG file.")
                return

            print(f"Metadata for {os.path.basename(file_path)}:")
            
            # Basic image information
            print(f"  Format: {img.format}")
            print(f"  Mode: {img.mode}")
            print(f"  Size: {img.size}")
            
            # Display all EXIF data
            exif_data = img._getexif()
            if exif_data:
                print("  EXIF Data:")
                for tag_id, value in exif_data.items():
                    tag = TAGS.get(tag_id, tag_id)
                    print(f"    {tag}: {value}")
            else:
                print("  No EXIF data found")
            
            # Other metadata
            print("  Other metadata:")
            for key, value in img.info.items():
                if key != 'exif':  # Skip exif data as we've already displayed it
                    print(f"    {key}: {value}")

            # If there's no additional metadata
            if not img.info and not exif_data:
                print("    No additional metadata found.")

    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")

def process_directory(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith('.png'):
            file_path = os.path.join(directory, filename)
            read_png_metadata(file_path)
            print()  # Empty line for separation between files

if __name__ == "__main__":
    path = r"P:\datasets\Posters\base\img_013.png"
    
    if os.path.isfile(path):
        read_png_metadata(path)
    elif os.path.isdir(path):
        process_directory(path)
    else:
        print(f"Error: {path} is not a valid file or directory.")

In [ ]:
import os
import torch
from diffusers import AutoencoderTiny
import safetensors
from PIL import Image
import matplotlib.pyplot as plt

def load_vae(device='cuda' if torch.cuda.is_available() else 'cpu'):
    vae = AutoencoderTiny.from_pretrained("madebyollin/taef1", torch_dtype=torch.bfloat16).to(device)
    return vae

def load_latent(path):
    try:
        if path.endswith('.safetensors'):
            with safetensors.safe_open(path, framework="pt", device="cpu") as f:
                latent = f.get_tensor('latent')
        elif path.endswith('.pt'):
            latent = torch.load(path, map_location='cpu')
            if isinstance(latent, dict) and 'latent' in latent:
                latent = latent['latent']
        else:
            raise ValueError(f"Unsupported file format: {path}")
        return latent
    except Exception as e:
        print(f"Error loading latent from {path}: {str(e)}")
        return None

def decode_latents(latents: torch.Tensor, vae: AutoencoderTiny):
    latents = latents.to(vae.device, dtype=torch.bfloat16)
    latents = latents / vae.config.scaling_factor
    with torch.no_grad():
        images = vae.decode(latents).sample
    return images

def tensor_to_pil(img):
    img = (img / 2 + 0.5).clamp(0, 1)
    img = img.cpu().float().permute(1, 2, 0).numpy()
    return Image.fromarray((img * 255).astype("uint8"))

def process_latent_file(file_path, vae, output_dir):
    print(f"Processing: {file_path}")
    latent = load_latent(file_path)
    if latent is not None:
        decoded_images = decode_latents(latent.unsqueeze(0), vae)
        for i, img_tensor in enumerate(decoded_images):
            img = tensor_to_pil(img_tensor)
            output_filename = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(file_path))[0]}_{i}.png")
            img.save(output_filename)
            print(f"Saved: {output_filename}")

def process_directory(input_dir, vae):
    output_dir = os.path.join(input_dir, "decoded")
    os.makedirs(output_dir, exist_ok=True)
    
    for file in os.listdir(input_dir):
        if file.endswith(('.safetensors', '.pt')):
            process_latent_file(os.path.join(input_dir, file), vae, output_dir)

if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    vae = load_vae(device)
    
    input_dir = r"P:\downloads\latents (5)\imagecache"
    process_directory(input_dir, vae)

In [2]:
!pip show diffusers  
!pip show accelerate

Name: diffusers
Version: 0.25.0
Summary: State-of-the-art diffusion in PyTorch and JAX.
Home-page: https://github.com/huggingface/diffusers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/diffusers/graphs/contributors)
Author-email: patrick@huggingface.co
License: Apache 2.0 License
Location: g:\anaconda3\envs\aes\lib\site-packages
Requires: filelock, huggingface-hub, importlib-metadata, numpy, Pillow, regex, requests, safetensors
Required-by: lycoris_lora


Name: accelerate
Version: 0.25.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: g:\anaconda3\envs\aes\lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft
